# Answering Business Questions using SQL|

## Conectando com o SQL

In [11]:
%%capture
%load_ext sql
%sql sqlite:///chinook.db

'Connected: None@chinook.db'

## Analisando DB

In [12]:
%%sql
SELECT
    name,
    type
FROM sqlite_master
WHERE type IN ("table","view");

Done.


[('album', 'table'),
 ('artist', 'table'),
 ('customer', 'table'),
 ('employee', 'table'),
 ('genre', 'table'),
 ('invoice', 'table'),
 ('invoice_line', 'table'),
 ('media_type', 'table'),
 ('playlist', 'table'),
 ('playlist_track', 'table'),
 ('track', 'table')]

In [15]:
%%sql

SELECT * FROM genre LIMIT 5

Done.


[(1, 'Rock'),
 (2, 'Jazz'),
 (3, 'Metal'),
 (4, 'Alternative & Punk'),
 (5, 'Rock And Roll')]

In [16]:
%%sql

SELECT * FROM customer LIMIT 5

Done.


[(1, 'Luís', 'Gonçalves', 'Embraer - Empresa Brasileira de Aeronáutica S.A.', 'Av. Brigadeiro Faria Lima, 2170', 'São José dos Campos', 'SP', 'Brazil', '12227-000', '+55 (12) 3923-5555', '+55 (12) 3923-5566', 'luisg@embraer.com.br', 3),
 (2, 'Leonie', 'Köhler', None, 'Theodor-Heuss-Straße 34', 'Stuttgart', None, 'Germany', '70174', '+49 0711 2842222', None, 'leonekohler@surfeu.de', 5),
 (3, 'François', 'Tremblay', None, '1498 rue Bélanger', 'Montréal', 'QC', 'Canada', 'H2G 1A7', '+1 (514) 721-4711', None, 'ftremblay@gmail.com', 3),
 (4, 'Bjørn', 'Hansen', None, 'Ullevålsveien 14', 'Oslo', None, 'Norway', '0171', '+47 22 44 22 22', None, 'bjorn.hansen@yahoo.no', 4),
 (5, 'František', 'Wichterlová', 'JetBrains s.r.o.', 'Klanova 9/506', 'Prague', None, 'Czech Republic', '14700', '+420 2 4172 5555', '+420 2 4172 5555', 'frantisekw@jetbrains.com', 4)]

## Total de vendas por gênero - USA

In [104]:
%%sql

WITH
    usa AS
        (
        SELECT *
        FROM customer
        WHERE country = 'USA'
        ),
    genre_total AS
        (
        SELECT
            inv.invoice_id,
            inv.customer_id,
            g.name genre_name,
            inv.total total_purchased
        FROM invoice inv
        INNER JOIN invoice_line invl ON inv.invoice_id = invl.invoice_line_id
        INNER JOIN track t ON invl.invoice_line_id = t.track_id
        LEFT JOIN genre g ON t.genre_id = g.genre_id
        ),
    consolidated AS
        (
        SELECT SUM(total) total_consolidated FROM invoice
        )

SELECT 
    gt.genre_name, 
    ROUND(CAST(SUM(gt.total_purchased) AS FLOAT), 3) total_purchased,
    ROUND(CAST(SUM(gt.total_purchased) AS FLOAT)/
        (
            SELECT SUM(total) total_consolidated FROM invoice
        ), 5) percent,
    usa.country
FROM genre_total gt
INNER JOIN usa ON gt.customer_id = usa.customer_id
GROUP BY 1
ORDER BY 3 DESC;

Done.


[('Rock', 330.66, 0.07021, 'USA'),
 ('Latin', 263.34, 0.05592, 'USA'),
 ('Alternative & Punk', 114.84, 0.02439, 'USA'),
 ('Reggae', 87.12, 0.0185, 'USA'),
 ('Metal', 75.24, 0.01598, 'USA'),
 ('Jazz', 74.25, 0.01577, 'USA'),
 ('Blues', 34.65, 0.00736, 'USA'),
 ('Soundtrack', 32.67, 0.00694, 'USA'),
 ('Pop', 26.73, 0.00568, 'USA'),
 ('Rock And Roll', 0.99, 0.00021, 'USA')]

É possível observar  que Rock e Latin são os gêneros mais consumidos pelo país. Ironicamente o Rock And Roll é o último da lista.

## Total vendido por Agente 

In [67]:
%%sql
SELECT * FROM employee WHERE title = "Sales Support Agent";


Done.


[(3, 'Peacock', 'Jane', 'Sales Support Agent', 2, '1973-08-29 00:00:00', '2017-04-01 00:00:00', '1111 6 Ave SW', 'Calgary', 'AB', 'Canada', 'T2P 5M5', '+1 (403) 262-3443', '+1 (403) 262-6712', 'jane@chinookcorp.com'),
 (4, 'Park', 'Margaret', 'Sales Support Agent', 2, '1947-09-19 00:00:00', '2017-05-03 00:00:00', '683 10 Street SW', 'Calgary', 'AB', 'Canada', 'T2P 5G3', '+1 (403) 263-4423', '+1 (403) 263-4289', 'margaret@chinookcorp.com'),
 (5, 'Johnson', 'Steve', 'Sales Support Agent', 2, '1965-03-03 00:00:00', '2017-10-17 00:00:00', '7727B 41 Ave', 'Calgary', 'AB', 'Canada', 'T3B 1Y7', '1 (780) 836-9987', '1 (780) 836-9543', 'steve@chinookcorp.com')]

In [77]:
%%sql

WITH
    total_cliente AS
        (
        SELECT 
            c.customer_id,
            c.support_rep_id,
            SUM(inv.total) total_sales
        FROM customer c
        INNER JOIN invoice inv ON c.customer_id = inv.customer_id
        GROUP BY 2
        )
        
SELECT 
    e.first_name || ' ' || e.last_name salesman,
    ROUND(tc.total_sales, 0)
FROM total_cliente tc
INNER JOIN employee e ON tc.support_rep_id = employee_id
ORDER BY 2 DESC;

Done.


[('Jane Peacock', 1732.0),
 ('Margaret Park', 1584.0),
 ('Steve Johnson', 1394.0)]

É possível observar que Jane é a melhor vendedora , seguida por Margaret e Steve respectivamente. A diferença entre as vendas dos três é relativamente próxima. 

## Analisando clientes por países

In [150]:
%%sql
DROP VIEW customer_country;

Done.


[]

In [151]:
%%sql

CREATE VIEW customer_country AS
    SELECT 
        COUNT(c.customer_id) AS customer_count,
        ROUND(inv.total_purchased, 0) AS total_purchased, 
        c.country
    FROM customer c
    INNER JOIN (
                SELECT 
                    customer_id,
                    SUM(total) total_purchased
                FROM invoice
                GROUP BY 1) inv ON c.customer_id = inv.customer_id
    GROUP BY 3;


SELECT * FROM customer_country LIMIT 5;

Done.
Done.


[(1, 40.0, 'Argentina'),
 (1, 81.0, 'Australia'),
 (1, 69.0, 'Austria'),
 (1, 60.0, 'Belgium'),
 (5, 107.0, 'Brazil')]

In [172]:
%%sql

DROP VIEW customer_country_analisys;

Done.


[]

In [173]:
%%sql

CREATE VIEW customer_country_analisys AS
    SELECT
        SUM(cc.customer_count) total_customers,
        SUM(cc.total_purchased) total_purchased,
    ROUND(cc.total_purchased/cc.customer_count, 0) sales_per_customer,
    CASE
        WHEN cc.customer_count = 1 THEN "Other"
        ELSE cc.country END 'country'
    FROM customer_country cc
    GROUP BY 4
    ORDER BY 3 DESC;

SELECT * FROM customer_country_analisys;

Done.
Done.


[(15, 1093.0, 75.0, 'Other'),
 (2, 129.0, 65.0, 'Czech Republic'),
 (2, 82.0, 41.0, 'Portugal'),
 (2, 71.0, 36.0, 'India'),
 (3, 79.0, 26.0, 'United Kingdom'),
 (5, 107.0, 21.0, 'Brazil'),
 (4, 73.0, 18.0, 'Germany'),
 (5, 73.0, 15.0, 'France'),
 (8, 75.0, 9.0, 'Canada'),
 (13, 72.0, 6.0, 'USA')]

É possível observar que os clientes dos países que tem apenas um cliente ('Other') tem um consumo acima da média e movimentam boa parte do lucro total. 

In [181]:
%%sql

SELECT 
ROUND(AVG(total_purchased), 4) AS avg_total_purchased,
ROUND(AVG(sales_per_customer), 4) AS avg_sales_per_customer
FROM customer_country_analisys;

Done.


[(185.4, 31.2)]

In [159]:
%%sql

SELECT country, customer_count 
FROM customer_country
WHERE customer_count = 1;

Done.


[('Argentina', 1),
 ('Australia', 1),
 ('Austria', 1),
 ('Belgium', 1),
 ('Chile', 1),
 ('Denmark', 1),
 ('Finland', 1),
 ('Hungary', 1),
 ('Ireland', 1),
 ('Italy', 1),
 ('Netherlands', 1),
 ('Norway', 1),
 ('Poland', 1),
 ('Spain', 1),
 ('Sweden', 1)]

Também conseguimos observar que:
    - O país que mais consome individualmente é a Czech Republic
    - O país que menos consome no valor absoluto é a India
    - O país com o menor consumo por cliente é os USA. 